In [1]:
# !pip install python==3.9
!pip install torch
!pip install deepparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 5.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 51.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.5/213.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 54.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.76
    Uninstalling botocore-1.29.76:
      Successfully uninstalled botocore-1.29.76
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency c

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
# import pytorch
from deepparse.parser import AddressParser
import deepparse


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [38]:
def generate_rand_garbage(list_bloc):
    list_garbage = ['',' ','École','Bibliothèque','Musée','Bureau de poste','Banque','Hôtel','Restaurant','Café','Cinéma','Théâtre',
                    'Stade','Gymnase','Parc aquatique','Magasin de vêtements','Pharmacie','Épicerie','Salon de coiffure','Station service',
                    'Garage','Boutique de jouets','Galerie d art','Studio de danse','Salle de concert','Centre commercial','Boulangerie',
                    'Université','Poste de police','Caserne de pompiers','Église','Mosquée','Synagogue','Temple','Hôpital','Clinique','Cabinet médical',
                    'Pharmacie','Maison de retraite','Centre de réadaptation','Salle de conférence','Salle d exposition']
    garbage = random.choice(list_garbage)
    list_extension = ['', ' de ' + list_bloc[4], ' van ' + list_bloc[4]]
    extension = random.choice(list_extension)
    if garbage == '': 
        return ''
    else : 
        return garbage + extension

def del_symbols(list_bloc):
    result = []
    symbols = "!@#$%^&*()_-+={}[]|\:;'<>?,./\""
    for item in list_bloc:
        new_item = ''.join([c if c.isalnum() or c.isspace() else ' ' for c in item])
        result.append(new_item)
    return result

def generate_rand_presence(list_bloc):
    N = len(list_bloc)
    random_list = [random.choices([0,1], [0.2,0.8])[0] for _ in range(N)]
    return random_list

def generate_rand_order(list_bloc):
    N = len(list_bloc)
    unique_list = list(range(1, N + 1))
    random.shuffle(unique_list)
    return unique_list

def add_typo_error(list_bloc, list_order):
    result = []
    error_type = random.choice(['nothing','add', 'delete'])
    for i in range(len(list_bloc)):
        string = list_bloc[i]
        order = list_order[i]
        if string is None:
            pass
        elif error_type == 'nothing' or order=='PostalCode' or len(string)<1: 
            pass
        elif error_type == 'add':
            error_index = random.randint(0, len(string))
            error_char = random.choice('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')
            string = string[:error_index] + error_char + string[error_index:]
        elif error_type == 'delete' and len(string)>1:
            if len(string) > 0:
                error_index = random.randint(0, len(string) - 1)
                string = string[:error_index] + string[error_index+1:]
        result.append(string)
    return result

def generate_elem_category(list_bloc, list_order = ['StreetNumber','StreetName','Unit','PostalCode','Municipality','GeneralDelivery']):
    list_elem_category = []
    if len(list_bloc) != len(list_order):
        print('il n\'y a pas autant de blocs que de categories ' + str(len(list_bloc)) + ' != ' + str(len(list_order)))
        raise 'il n\'y a pas autant de blocs que de categories ' + str(len(list_bloc)) + ' != ' + str(len(list_order))
    for i in range(len(list_bloc)):
        bloc = list_bloc[i]
        order = list_order[i]
        len_bloc = len(bloc.split())
        elem_category = [order] * len_bloc
        list_elem_category.append(elem_category)
    return list_elem_category

def change_list_order(lst, order):
    if len(lst) != len(order):
        raise ValueError("List and order must have the same length.")
    new_lst = [None] * len(lst)
    for i, value in enumerate(lst):
        new_index = order[i] - 1  
        new_lst[new_index] = value
    return new_lst

def address_augmentation(list_bloc, list_order):
    
    list_bloc[-1] = generate_rand_garbage(list_bloc)
    # print(list_bloc)
    list_bloc = del_symbols(list_bloc)
    # print(list_bloc)
    list_bloc = add_typo_error(list_bloc, list_order)
    # print(list_bloc)
    list_elem_cat = generate_elem_category(list_bloc)
    rand_presence = generate_rand_presence(list_bloc)
    rand_order = generate_rand_order(list_bloc)
    
    list_bloc_ordered = change_list_order(list_bloc,rand_order)
    list_elem_cat_ordered = change_list_order(list_elem_cat,rand_order)
    rand_presence_ordered = change_list_order(rand_presence,rand_order)
    
    list_final_category = []
    string_final = ''
    for i in range(len(rand_presence_ordered)):
        if rand_presence_ordered[i] == 1:
            if len(list_bloc_ordered[i].strip()) != 0 or list_bloc_ordered[i] is not None: 
                list_final_category.append(list_elem_cat_ordered[i])
            if i == 0:
                string_final = list_bloc_ordered[0]
            else :
                string_final = string_final + ' ' + list_bloc_ordered[i]
    string_final = string_final
    flattened_list_final_category = [element for sublist in list_final_category for element in sublist]
    
    return string_final, flattened_list_final_category

In [8]:
!pip install pytest
!pip install mock

In [39]:
from unittest import mock

# ['StreetName','StreetNumber','Unit','PostalCode','Municipality','GeneralDelivery']
list_bloc1 = ['86','Avenue des Camélias','','1150','Woluwé-Saint-Pierre','']
list_bloc2 = ['22','Belgielaan','','1800','Vilvoorde','']
list_bloc3 = ['115','Rue inventée','4','1200','Woluwé-Saint-Lambert','Boutique de jouets']
list_bloc4 = ['8','Avenue de Boileau','5','1160','Auderghem','Magasin de vêtements de Auderghem']
list_bloc5 = ['8','Avenue de Boileau','5','1160','Auderghem','']
list_bloc6 = ['115','Rue inventée','4','1200','Woluwé-Saint-Lambert','']

# test part
def test_generate_rand_garbage():
    with mock.patch('random.choice') as mock_choice:
        mock_choice.side_effect = ['',' de ' + list_bloc1[4]]
        g = generate_rand_garbage(list_bloc1)
        assert g == '', g
    with mock.patch('random.choice') as mock_choice:
        mock_choice.side_effect = ['Bibliothèque', ' de ' + list_bloc2[4]]
        g = generate_rand_garbage(list_bloc2)
        assert g == 'Bibliothèque de Vilvoorde', g

def test_del_symbols():
    g1 = del_symbols(list_bloc1)
    assert g1 == ['86','Avenue des Camélias','','1150','Woluwé Saint Pierre',''], g1
    g2 = del_symbols(list_bloc6)
    assert g2 == ['115','Rue inventée','4','1200','Woluwé Saint Lambert',''], g2

def test_generate_rand_presence():
    g1 = generate_rand_presence(list_bloc1)
    assert sum(g1) <= 6, g1
    g2 = generate_rand_presence(list_bloc2)
    assert sum(g2) >= 0, g2

def test_generate_rand_order():
    g = generate_rand_order(list_bloc3)
    g_sorted = sorted(g)
    assert g_sorted == [1,2,3,4,5,6], g

'''
def test_add_typo_error():
    with mock.patch('random.randint') as mock_index, \
         mock.patch('random.choice') as mock_choice:  
        mock_index.return_value = 5
        mock_choice.return_value = 'delete'
        assert add_typo_error(list_bloc) == 
'''

# ['8','Avenue de Boileau','5','1160','Auderghem','Magasin de vêtements de Auderghem']
def test_generate_elem_category():
    g = generate_elem_category(list_bloc4, list_order = ['StreetNumber','StreetName','Unit','PostalCode','Municipality','GeneralDelivery'])
    assert g == \
        [['StreetNumber'],['StreetName','StreetName','StreetName'],['Unit'],['PostalCode'],['Municipality'],['GeneralDelivery','GeneralDelivery','GeneralDelivery','GeneralDelivery','GeneralDelivery']], g

"""
def test_change_list_order():
    assert change_list_order(lst, order) == 

def test_address_augmentation():
    assert address_augmentation(list_bloc) == 
"""

test_generate_rand_garbage()
test_del_symbols()
test_generate_rand_presence()
test_generate_rand_order()
test_generate_elem_category()

In [48]:
list_add = [list_bloc1,list_bloc2,list_bloc3,list_bloc4,list_bloc5,list_bloc6]
list_order = ['StreetNumber','StreetName','Unit','PostalCode','Municipality','GeneralDelivery']
print(list_add)
for s in list_add:    
    # print(s)
    # print(type(s))
    # print(len(s))
    print(address_augmentation(s,list_order))

[['86', 'Avenue des Camélias', '', '1150', 'Woluwé-Saint-Pierre', ''], ['22', 'Belgielaan', '', '1800', 'Vilvoorde', ''], ['115', 'Rue inventée', '4', '1200', 'Woluwé-Saint-Lambert', 'Boutique de jouets'], ['8', 'Avenue de Boileau', '5', '1160', 'Auderghem', 'Magasin de vêtements de Auderghem'], ['8', 'Avenue de Boileau', '5', '1160', 'Auderghem', ''], ['115', 'Rue inventée', '4', '1200', 'Woluwé-Saint-Lambert', '']]
('1150 6 Avenue de Camélias Hôtel de Woluwé Saint Pirre', ['PostalCode', 'StreetNumber', 'StreetName', 'StreetName', 'StreetName', 'GeneralDelivery', 'GeneralDelivery', 'GeneralDelivery', 'GeneralDelivery', 'GeneralDelivery'])
('Belgielaan Vilvoorde  Centre de réadaptation de Vilvoorde', ['StreetName', 'Municipality', 'GeneralDelivery', 'GeneralDelivery', 'GeneralDelivery', 'GeneralDelivery', 'GeneralDelivery'])
(' 115 Centre commercial de Woluwé Saint Lambert 4 Rue inventée', ['StreetNumber', 'GeneralDelivery', 'GeneralDelivery', 'GeneralDelivery', 'GeneralDelivery', 'Gen

Dataset Preprocessing part 1: join muni and best

In [46]:
df_muni = pd.read_csv('/kaggle/input/municipalities/muni.csv')
df_muni = df_muni.astype(str)
df_muni = df_muni.replace('nan', None)
df_muni.columns = 'muni_' + df_muni.columns
df_muni['muni_validity_from'] = pd.to_datetime(df_muni['muni_validity_from'])
df_muni = df_muni.sort_values('muni_validity_from', ascending=False)
df_muni = df_muni.drop_duplicates(subset='muni_muni_code',keep='first').reset_index()
print(df_muni)

# df_muni = df_muni[['muni_muni_code','muni_name_fr','muni_name_nl', 'muni_name_de']]

df_muni_fr= df_muni[['muni_muni_code','muni_name_fr']]
df_muni_fr.columns = ['muni_code','muni_name']

df_muni_nl= df_muni[['muni_muni_code','muni_name_nl']]
df_muni_nl.columns = ['muni_code','muni_name']

df_muni_de= df_muni[['muni_muni_code','muni_name_de']]
df_muni_de.columns = ['muni_code','muni_name']

del df_muni
df_muni = df_muni_fr
del df_muni_fr
df_muni = pd.concat([df_muni, df_muni_nl], axis=0)
del df_muni_nl
df_muni = pd.concat([df_muni, df_muni_de], axis=0)
del df_muni_de

df_muni.dropna(subset = ['muni_name'], inplace=True)
df_muni.drop_duplicates(subset = ['muni_name'], inplace=True)

print(df_muni)

     index muni_sid muni_muni_code       muni_name_fr       muni_name_nl  \
0     3501     3501          93090          Viroinval          Viroinval   
1     3138     3113          32010          Koekelare          Koekelare   
2     3118     3094          25118           Hélécine           Hélécine   
3     3117     3093          25117            Chastre            Chastre   
4     3116     3092          25112              Wavre              Waver   
..     ...      ...            ...                ...                ...   
602     24     1081          72040  Meeuwen-Gruitrode  Meeuwen-Gruitrode   
603     19     1076          72029           Overpelt           Overpelt   
604   1037      628          12030              Puurs              Puurs   
605   1038      629          12034        Sint-Amands        Sint-Amands   
606   1560     1063          71047         Opglabbeek         Opglabbeek   

          muni_name_de muni_validity_from muni_validity_to  \
0            Viroinval   

In [47]:
df_best = pd.read_csv('/kaggle/input/best-address/best.csv')
df_best = df_best.astype(str)
df_best = df_best.replace('nan', None)
df_best.columns = 'best_' + df_best.columns

df_best = df_best[['best_streetname_fr', 'best_streetname_nl',
                  'best_streetname_de', 'best_house_number', 'best_box_number', 'best_muni_code', 'best_poco_code']]

df_best_fr = df_best[['best_streetname_fr','best_house_number', 'best_box_number', 'best_muni_code', 'best_poco_code']]
df_best_fr.columns = ['best_streetname','best_house_number', 'best_box_number', 'best_muni_code', 'best_poco_code']
df_best_nl = df_best[['best_streetname_nl','best_house_number', 'best_box_number', 'best_muni_code', 'best_poco_code']]
df_best_nl.columns = ['best_streetname','best_house_number', 'best_box_number', 'best_muni_code', 'best_poco_code']
df_best_de = df_best[['best_streetname_de','best_house_number', 'best_box_number', 'best_muni_code', 'best_poco_code']]
df_best_de.columns = ['best_streetname','best_house_number', 'best_box_number', 'best_muni_code', 'best_poco_code']

df_best = df_best_fr
del df_best_fr
df_best = pd.concat([df_best, df_best_nl], axis=0)
del df_best_nl
df_best = pd.concat([df_best, df_best_de], axis=0)
del df_best_de

df_best.dropna(subset = ['best_streetname'], inplace=True)
df_best.drop_duplicates(subset = ['best_streetname','best_house_number','best_box_number','best_muni_code','best_poco_code'], inplace=True)

print(df_best)


/tmp/ipykernel_32/3657723589.py:1: DtypeWarning: Columns (2,3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_best = pd.read_csv('/kaggle/input/best-address/best.csv')


                best_streetname best_house_number best_box_number  \
0                    Rue Rasson                87            None   
1           Avenue Maréchal Ney                55            None   
2            Avenue du Centaure                 2            None   
3        Avenue Henri Hollevoet                14            None   
4                Rue Vandeweyer                43            None   
...                         ...               ...             ...   
6555289             Mühlengasse                17            None   
6555535              Zur Domäne                51            None   
6555536              Zur Domäne                75            None   
6555546              Zur Domäne                33            None   
6555550              Zur Domäne                 7               A   

        best_muni_code best_poco_code  
0                21015           1030  
1                21016           1180  
2                21018           1200  
3          

In [50]:
address = pd.merge(df_best,df_muni,left_on=['best_muni_code'],right_on=['muni_code'])

print(address)

             best_streetname best_house_number best_box_number best_muni_code  \
0                 Rue Rasson                87            None          21015   
1                 Rue Rasson                87            None          21015   
2             Rue Vandeweyer                43            None          21015   
3             Rue Vandeweyer                43            None          21015   
4         Rue Général Gratry                49            None          21015   
...                      ...               ...             ...            ...   
10443588             Am Bach                 5               1          63048   
10443589       Bahnhofstraße                 2            B/16          63048   
10443590      Schloss Straße                21            000C          63048   
10443591       Bahnhofstraße                 2             B/5          63048   
10443592         Schulstraße                15            None          63048   

         best_poco_code mun

In [51]:
print(address.dtypes)
test = address.iloc[0:2].values.tolist()
print(test)
# print(np.isnan(test[0][2]))

best_streetname      object
best_house_number    object
best_box_number      object
best_muni_code       object
best_poco_code       object
muni_code            object
muni_name            object
dtype: object
[['Rue Rasson', '87', None, '21015', '1030', '21015', 'Schaerbeek'], ['Rue Rasson', '87', None, '21015', '1030', '21015', 'Schaarbeek']]


Dataset Preprocessing part 2: Augmentation

In [ ]:
list_address = address.values.tolist()
print(list_address)
list_2_container = []

for add in list_address:
    inp, out = address_augmentation(s,list_order)
    list_2_container.append([inp, out])

In [ ]:
print(list_address[2])

Dataset Preprocessing part 3: list to container

In [ ]:


container_train = deepparse.dataset_container.ListDatasetContainer(list_train)

In [ ]:
address_parser = AddressParser(model_type="bpemb", device=0)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session